In [3]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

In [5]:
tf.keras.backend.set_floatx('float64')

In [11]:
train = pd.read_csv('energy/train.csv',encoding='cp949')
test = pd.read_csv('energy/test.csv', encoding='cp949')
submission = pd.read_csv('energy/sample_submission.csv', encoding='cp949')

In [12]:
train.head(5)

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0


In [13]:
train.shape

(122400, 10)

60개의 건물 X 85 일 X 24시간 == 122400    
Q. 왜 60개 건물을 한꺼번에 모은거지?  
data-time이 연 월 일 시 인 것 같은데,   
  
85일 x 24시간 단위로 1 건물씩 나눠질꺼란 얘긴데... 
A. 데이터 중 num이 건물 number 즉, power_plant의 건물 num이다
  
Q1.1    
1건물당 60개 단위로 어떻게 쪼개지는지 확인해보자  
그러니까 1번부터 60번까지 데이터들을 하나의 묶음처럼 통합해서 써도 되는건지  
아니면 1번 따로 2번 따로 해서 나중에 앙상블 하는 형식처럼 사용할 것인지   
  
  
Q2.
이 데이터로 그대로 진행해도 되는지 (ex/ 잘 정규화 되었는지)




In [16]:
print(train.iloc[:,2].max())

60


In [18]:
#전력사용량 정규화
mini = train.iloc[:,2].min()
print(mini)
size = train.iloc[:,2].max() - train.iloc[:,2].min() #걍 max하고 빼면 되는거 아닌가? 그보다 왜 최대에서 최소를 뺀거지?
print(size)
train.iloc[:,2] = (train.iloc[:,2] - mini) / size #이렇게 하면 모두다 0~1 사이로 정규화된다.

0.0
17739.225


In [21]:
train.head(5)

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,0.461072,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,0.458624,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,0.457017,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,0.453729,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,0.453437,17.0,3.3,92.0,0.0,0.0,0.0,0.0


In [43]:
#PARAMS
input_window = 996 # 임의의 수??
output_window = 24 #168 7일 24시간
window = 12 #window는 12시간 마다는 12시간마다 뭔소리지???
num_features = 1 #베이스라인은 피쳐 하나 사용
num_power = 60
end_ = 168
lstm_units = 32
dropout= 0.2
EPOCHS = 30
BATCH_SIZE = 128

In [24]:
#train 을 tensor로 변경 
train_x = tf.reshape(train.iloc[:,2].values, [num_power, 24 * 85, num_features])
print(f'train_x.shape:{train_x.shape}')

train_x.shape:(60, 2040, 1)


(60, 24 * 85, 1) 인데, 가운데는 24시간을 85일로 나눠놓은 것

In [29]:
train_window_x = np.zeros((train_x.shape[0],
                          (train_x.shape[1] - (input_window + output_window)) // window ,
                         input_window,
                         num_features))
train_window_y = np.zeros((train_x.shape[0],
                          (train_x.shape[1] - (input_window + output_window)) // window ,
                         output_window,
                         num_features))
print(f'train_window_x.shape:{train_window_x.shape}')
print(f'train_window_y.shape:{train_window_y.shape}')

train_window_x.shape:(60, 85, 996, 1)
train_window_y.shape:(60, 85, 24, 1)


train_window_x np.zeros를 만듬 (60, 85, 996, 1)  
이 과정 왜 하는지 모르겠다 
zeros는 shape에 맞게 0을 넣어주는건데  
일종의 초기화 해놓는 과정같은건가?? 만약 초기화 과정이다 하더라도 그냥 만들면 되는거지 왜 초기화를 하는거지?

In [30]:
#train_window_x 에 train값 채워넣기
for example in range(train_x.shape[0]):
    for start in range(0, train_x.shape[1] - (input_window + output_window), window):
        end = start + input_window
        train_window_x[example, start//window, :] =train_x[example, start : end, :]
        train_window_y[example, start//window, :] =train_x[example, end : end + output_window, :]

In [31]:
#new_train_x, reshape을 통해 lstm에 알맞는 형태로 집어넣기
new_train_x = tf.reshape(train_window_x, [-1, input_window, num_features])
new_train_y = tf.reshape(train_window_y, [-1, output_window, num_features])
print(f'new_train_x.shape:{new_train_x.shape}')
print(f'new_train_y.shape:{new_train_y.shape}')

new_train_x.shape:(5100, 996, 1)
new_train_y.shape:(5100, 24, 1)


In [33]:
#층 쌓기
model = Sequential([
    LSTM(lstm_units, return_sequences=False, recurrent_dropout = dropout),
    Dense(output_window * num_features, kernel_initializer=tf.initializers.zeros()),
    Reshape([output_window, num_features])
])

In [37]:
model.compile(optimizer = 'rmsprop', loss='mae', metrics=['mae'])

In [39]:
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 10 == 0:
            print("")
        print('.',end="")

In [40]:
save_best_only = tf.keras.callbacks.ModelCheckpoint(filepath='lstm_model.h5', monitor='val_loss', save_best_only= True)

In [41]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [42]:
#검증 손실이 10 에폭동안 좋아지지 않으면 학습률을 0.1배로 재구성하는 명령어
reduce_LR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', facotr=0.1, patience=10)

In [45]:
model.fit(new_train_x,
         new_train_y,
         epochs=EPOCHS,
          batch_size = BATCH_SIZE,
          validation_split = 0.2,
          verbose = 0,
          callbacks=[PrintDot(), early_stopping, save_best_only, reduce_LR]
         )


..........
..........
..........

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 24)                792       
_________________________________________________________________
reshape (Reshape)            (None, 24, 1)             0         
Total params: 5,144
Trainable params: 5,144
Non-trainable params: 0
_________________________________________________________________


In [47]:
prediction = np.zeros((num_power, end_, num_features))
new_test_x = train_x

In [48]:
for i in range(end_//output_window):
    start_ = i * output_window
    next_ = model.predict(new_test_x[:, -input_window:, :])
    new_test_x = tf.concat([new_test_x, next_], axis=1)
    print(new_test_x.shape)
    prediction[:, start_:start_ + output_window, :] = next_
prediction = prediction * size + mini

(60, 2064, 1)
(60, 2088, 1)
(60, 2112, 1)
(60, 2136, 1)
(60, 2160, 1)
(60, 2184, 1)
(60, 2208, 1)


In [49]:
submission['answer'] = prediction.reshape([-1,1])

In [50]:
submission

,num_date_time,answer
0,1 2020-08-25 00,8099.731702
1,1 2020-08-25 01,8093.524296
2,1 2020-08-25 02,8011.479852
3,1 2020-08-25 03,8008.108204
4,1 2020-08-25 04,7978.737996
...,...,...
10075,60 2020-08-31 19,2413.436805
10076,60 2020-08-31 20,2300.989576
10077,60 2020-08-31 21,2095.198444
10078,60 2020-08-31 22,1920.271452


In [52]:
submission.to_csv('energy/sample_submission.csv', encoding='cp949', index=False)